# Imager Optimization

## Function Imports and Definitions

In [ ]:
import pyrayt
from pyrayt import components
import pyrayt.materials as matl
from pyrayt.utils import lensmakers_equation
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from typing import Tuple
from tinygfx.g3d import ObjectGroup
import pandas as pd
from tinygfx.g3d.renderers 
from tinygfx.g3d import ObjectGroup
mport draw

def init_figure() -> Tuple[plt.Figure, plt.Axes]:
    """
    Convenience function to generate an axis with a set size and 
    """
    fig = plt.figure(figsize = (12,8))
    axis = plt.gca()
    axis.grid()
    return fig, axis


## System Constants

In [ ]:
# All spatial units are mm
lens_diameter = 30
lens_thickness = 8
system_focus = 50 # The focus of the system
f_num = 2.4

## A Single Lens Imager

In [ ]:
# Creating a simple Lens 
lens_material = matl.glass"[BKl"]
lens_radius = 2*(lens_material.index_at(0.633)-1)*system_focus # We're discarding the lens thickness in the radius calculation
lens = pyrayt.components.thick_lens(
    r1=lens_radius, 
    r2=-lens_radius,
    thickness=lens_thickness,
    aperture=lens_diameter,
    material=lens_material)

draw(lens)

In [ ]:
# Creating the Imager
imager = components.baffle((lens_diameter, lens_diameter)).move_x(system_focus)

In [ ]:
# Creating the Aperture
aperture_position = system_focus / 2 # place it half way between the lens and the imager
aperture_diameter = aperture_position / f_num
aperture = components.aperture(
    size=(lens_diameter, lens_diameter), 
    aperture_size=aperture_diameter
    ).move_x(aperture_position)

In [ ]:
# Create a Parallel ray set
source = components.LineOfRays(0.8*lens_diameter, wavelength = 0.633).move_x(-10)

tracer = pyrayt.RayTracer(source, [lens, aperture, imager])
tracer.set_rays_per_source(11)
results = tracer.trace()
fig, axis = init_figure()

tracer.show(
    #color_function="source",
    ray_width=0.2,
    axis=axis,
    view='xy')
plt.show()

### Spherical Aberrations

In [ ]:
n_sources = 5
source_diameter = np.linspace(0.1, 0.5, n_sources)*lens_diameter
spherical_sources = [components.CircleOfRays(x).move_x(-10) for x in source_diameter]
tracer = pyrayt.RayTracer(spherical_sources, [lens, aperture, imager])
tracer.set_rays_per_source(111)
results = tracer.trace()
tracer.calculate_source_ids() # calculates which source generated each ray and adds it to the dataframe



imager_rays = results.loc[results['surface'] == imager.get_id()]

fig, axis = init_figure()
axis.set_aspect('equal')
axis.set_xlabel("y-axis (mm)")
axis.set_ylabel("z-axis (mm)")
axis.set_title("spot size at focus for various beam diameters")
for n, radii in enumerate(source_diameter):
    source_rays_on_imager = imager_rays.loc[imager_rays['source_id']==n]
    axis.plot(source_rays_on_imager['y1'], source_rays_on_imager['z1'],'-o',label=f"{radii:.02f}mm")
plt.legend()
plt.show()



In [ ]:
def spherical_aberration(system, ray_origin: float, max_radius:float, sample_points=11):

    # the souce is a line of rays only on the +y axis. It's slightly shifted so zero is not a point
    # as it would focus at infinity
    source = pyrayt.components.LineOfRays(0.9*max_radius).move_x(ray_origin).move_y(max_radius/2)


    tracer = pyrayt.RayTracer(source, system)
    tracer.set_rays_per_source(sample_points)
    results = tracer.trace()

    # Since we don't have the actual imager as a variable in the function
    # assume it is the last thing a ray intersect with, meaning the rays that hit it have the 
    # highest generation
    imager_rays = results.loc[results['generation'] == np.max(results['generation'])]
    
    # Intercept is calculated using the tilt for each ray, with is a normalized vector representing
    # the direction the ray is travelling
    intercept = -imager_rays['x_tilt']*imager_rays['y0']/imager_rays['y_tilt'] + imager_rays['x0']

    # the original radii 
    radii = results.loc[np.logical_and(results['generation']==0, results['id'].isin(imager_rays['id']))]['y0']

    # create a new dataframe with the aberration metrics
    results = pd.DataFrame({'radius': np.asarray(radii), 'focus': np.asarray(intercept)})
    return results



In [ ]:
aberrations = spherical_aberration([lens, aperture, imager], -10, 0.4*lens_diameter, 21)

fig, axis = init_figure()
axis.set_title("Focal Length vs. Beam Radius for a single-lens imager")
axis.plot(aberrations['radius'], aberrations['focus'])
axis.set_xlabel("Beam Radius (mm)")
axis.set_ylabel("Focal Length (mm)")
plt.show()

### Chromatic Aberrations

In [ ]:
def chromatic_abberation(system, ray_origin: float, test_radius:float, wavelengths: np.ndarray) -> pd.DataFrame:
    # create a set of sources for every wavelength of light
    sources = [
        pyrayt.components.LineOfRays(0, wavelength = wave)
        .move_y(test_radius)
        .move_x(ray_origin) 
        for wave in wavelengths]
    
    # Create the ray tracer and propagate
    tracer = pyrayt.RayTracer(sources, system)
    tracer.set_rays_per_source(1)
    results = tracer.trace()

    #filter the rays that intersect the imager
    imager_rays = results.loc[results['generation'] == np.max(results['generation'])]
    
    # calculate intercept of the imager rays with the x-axis and form into a dataframe
    intercept = -imager_rays['x_tilt']*imager_rays['y0']/imager_rays['y_tilt'] + imager_rays['x0']
    results = pd.DataFrame({'wavelength': imager_rays['wavelength'], 'focus': intercept})


    return results



In [ ]:
aberrations = chromatic_abberation([lens, aperture, imager], -10, lens_diameter/4, np.linspace(0.45, 0.75, 11))

fig, axis = init_figure()
axis.set_title("Focal Length vs. wavelength for a single-lens imager")
axis.plot(1000*aberrations['wavelength'], aberrations['focus'])
axis.set_xlabel("Beam Radius (mm)")
axis.set_ylabel("wavelength (nm)")
plt.show()

### Coma Aberration

In [ ]:
# Visualizing with three sources
angle_sources= [pyrayt.components.CircleOfRays(0.5 * lens_diameter).move_x(-10).rotate_z(x) for x in (0, 10, 15)]

tracer = pyrayt.RayTracer(angle_sources, [lens, imager])
tracer.set_rays_per_source(100)
tracer.trace()
fig, axis = init_figure()
tracer.show(
    axis=axis,
    color_function="source",
    ray_width=0.15
)

In [ ]:
# View the Coma distortion at each angle
tracer.calculate_source_ids()
results = tracer.get_results()

fig = plt.figure(figsize=(12,12))
for source in set(results['source_id']):
    ray_set = results.loc[np.logical_and(results['source_id']==source, results['surface']==imager.get_id())]
    axis = plt.subplot(2,2,source+1)
    axis.set_aspect('equal')
    axis.plot(ray_set['z1'], ray_set['y1'],'o')
    

In [ ]:
def coma(system, ray_origin: float, max_radius:float, angle: float) -> float:
    source = pyrayt.components.LineOfRays(2*max_radius).move_x(-10).rotate_z(angle)

    tracer = pyrayt.RayTracer(source, system)
    tracer.set_rays_per_source(11)
    results = tracer.trace()
    tracer.calculate_source_ids()

    ray_set = results.loc[results['surface'] == imager.get_id()]
    expected_y = system_focus*np.tan(angle*np.pi/180)
    return np.mean(ray_set['y1'] - expected_y)

In [ ]:
coma([lens, imager], -10, 0.4*lens_diameter, 0)

## Making a Doublet

In [ ]:
# Want the power of the whole system to remain unchanged, but cancel out for the first Order Chromatic Dispersion
# Define the two materials we will use
bk7tl = matl.glass["BK7 # abbe of ~60 - high disperion"sf5atl = matl.glass["SF # abbe of ~30 - low dispersion

# will make the first lens out of bk7 and the second out of sf5
# get abbe numbers for each material
v1 = bk7.abbe()
v2 = sf5.abbe()

# calculate the individual lens powers based on the dispersion
p1 = p_sys * v1/(v1 - v2)
p2 = p_sys * v2/(v2 - v1)

# calculate the lens radius from the lensmakers equation 
r1 = 2*(bk7.index_at(0.633)-1) / p1
r2 = 2*(sf5.index_at(0.633)-1) / p2s=axis)
plt.show()

In [ ]:
# create lenses as a doublet
lens_thickness=8
l1 = pyrayt.components.thick_lens(r1, -r1, lens_thickness, aperture = lens_diameter, material=bk7)
l2 = pyrayt.components.thick_lens(r2, -r2, 2, aperture = lens_diameter, material=sf5).move_x(1.001*lens_thickness/2+1)
doublet = ObjectGroup([l1, l2])

In [ ]:
source = pyrayt.components.LineOfRays(0.5*lens_diameter).move_x(-10).rotate_z(12)

tracer = pyrayt.RayTracer(source, [*doublet, aperture, imager])
tracer.trace()
tracer.show(
    ray_width=0.4

)

In [ ]:
aberrations = [chromatic_abberation([lens, aperture, imager], -10, lens_diameter/4, np.linspace(0.45, 0.75, 11)),
chromatic_abberation([*doublet, aperture, imager], -10, lens_diameter/4, np.linspace(0.45, 0.75, 11))]


fig, axis = init_figure()
axis.set_title("Focal Length vs. wavelength for a single-lens imager")
for aberration in aberrations:
    axis.plot(1000*aberration['wavelength'], aberration['focus']-np.mean(aberration['focus']))
axis.set_xlabel("Beam Radius (mm)")
axis.set_ylabel("wavelength (nm)")
plt.show()

In [ ]:
def test_source(wave):
    return pyrayt.components.LineOfRays(0, wavelength=wave).move_x(-10).move_y(lens_diameter/4)urn results

def spherical_abberation(source, system, sample_points=11):
    tracer = pyrayt.RayTracer(source, system)
    tracer.set_rays_per_source(sample_points)
    results = tracer.trace()
    imager_rays = results.loc[results['generation'] == np.max(results['generation'])]
    intercept = -imager_rays['x_tilt']*imager_rays['y0']/imager_rays['y_tilt'] + imager_rays['x0']
    radii = results.loc[results['generation']==0]['y0']
    results = pd.DataFrame({'radius': np.asarray(radii), 'focus': np.asarray(intercept)})
    return results


        

In [ ]:
abberation = chromatic_abberation([lens, imager], test_source, np.linspace(0.44, 0.75, 11))
abberation_doublet = chromatic_abberation([*doublet, imager], test_source, np.linspace(0.44, 0.75, 21))

fig, axis = init_figure()
axis.plot(1000*abberation['wavelength'], abberation['focus']-50)
axis.plot(1000*abberation_doublet['wavelength'], abberation_doublet['focus']-np.mean(abberation_doublet['focus']))
axis.set_xlabel("Wavelength (nm)")
axis.set_ylabel("Focal Length Shift (mm)")
plt.show()

In [ ]:
abberation = spherical_abberation(pyrayt.components.LineOfRays(aperture_diameter*0.4).move(-10,0.22*aperture_diameter), [lens, imager])
abberation_doublet = spherical_abberation(pyrayt.components.LineOfRays(aperture_diameter*0.4).move(-10,0.22*aperture_diameter), [*doublet, imager])

fig, axis = init_figure()
axis.plot(abberation['radius'], abberation['focus']-system_focus)
axis.plot(abberation_doublet['radius'], abberation_doublet['focus']-np.mean(abberation_doublet['focus']))
axis.set_xlabel("Wavelength (nm)")
axis.set_ylabel("Focal Length Shift (mm)")
plt.show()

In [ ]:
results.loc[results['generation'] == np.max(results['generation'])]

In [ ]:
tracer = pyrayt.RayTracer(source, [optimized_lens, aperture, imager])
tracer.set_rays_per_source(11)
results = tracer.trace()
fig, axis = init_figure()
tracer.show(
    ray_width=0.1,
    axis=axis)
plt.show()

fig, axis = init_figure()
tracer.show(
    ray_width=0.025,
    axis=axis)
axis.grid()
axis.set_xlim(system_focus - 1, system_focus + 1)
axis.set_ylim(-1, 1)
plt.show()

### Chromatic Abberation

In [ ]:
sources = [components.LineOfRays(0.3*lens_diameter, wavelength = x).move_x(-system_focus).rotate_z(0) for x in [0.44, 0.53, 0.65]]
tracer = pyrayt.RayTracer(sources, [optimized_lens, aperture, imager.move_x(3)])
tracer.set_rays_per_source(2)
results = tracer.trace()
fig, axis = init_figure()
tracer.show(
    color_function="wavelength",
    ray_width=0.1,
    axis=axis)
plt.show()

fig, axis = init_figure()
tracer.show(
    color_function="wavelength",
    ray_width=0.025,
    axis=axis)
axis.grid()
axis.set_xlim(system_focus - .5, system_focus + 1)
axis.set_ylim(-.5, 0.5)
plt.show()